In [11]:
import quandl
import datetime
import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from Robinhood import Order
from Robinhood import WatchList
from Robinhood import Robinhood
from threading import Thread
from threading import Lock
from time import sleep
from pytz import timezone

%matplotlib inline

In [32]:
class Portfolio:
    def __init__(
        self,
        trader = None,
        name = None,
        iniFund = None,
        load_from = None,
        cancel_count = np.inf
    ):
        """
        create portfolio or load from save
        trader (Robinhood): robinhood account trader
        name (str): name of this portfolio
        iniFund (float|None): initial buying power for this portfolio
        load_from (str|None): address to save information
        """
        assert trader is not None
        assert name is not None
        self.name = name
        self.trader = trader
        self.bp = iniFund
        self.confirm_signal = True
        self.threads = []
        if iniFund == None or iniFund >= float(trader.get_account()['margin_balances']['unallocated_margin_cash'])*0.7:
            self.bp = float(trader.get_account()['margin_balances']['unallocated_margin_cash'])*0.7
            
            
        self.trading_record = pd.DataFrame(columns = ["SIDE","SCODE","PRICE","AMOUNT","ORDER_TYPE"])
        self.trading_record_lock = Lock()
        
        self.portfolio_record = pd.DataFrame(columns = ['AVG_COST','SHARES'])
        self.portfolio_record_lock = Lock()
        
        self.queue = []
        self.queue_lock = Lock()
        
        self.time_zone = str(datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo)
        
        self.log = []
        self.log_lock = Lock()
        
        self.cancel_count = cancel_count
        if load_from is not None:
            self.load(load_from)
    def get_last_price(self):
        """
        get last trading price for any asset in this portfolio
        """
        self.portfolio_record_lock.acquire()
        res = np.array(
            trader.last_trade_price(','.join(self.portfolio_record.index))
        )[:,0].astype(np.float32)
        self.portfolio_record_lock.release()
        return res
    
    def get_time(self):
        """
        get current time
        """
        return pd.datetime.now()
    
    def get_market_value(self):
        """
        get market value of current portfolio
        """
        self.portfolio_record_lock.acquire()
        res = np.dot(
            self.get_last_price(),
            self.portfolio_record["SHARES"].values
        )+self.bp
        self.portfolio_record_lock.release()
        return res
    
    def market_buy(self,scode,n,force_buy = False,time_in_force = 'gfd'):
        """
        buy stock with market order
        
        scode (str): symbol of stock
        n (int): shares to buy
        force_buy (bool): allow using back up buying power(exception when there isnt any) to buy
        time_in_force (str): gfd ,gtc ,ioc ,fok or opg
        """
        def market_buy_worker():
            if self.bp < float(self.trader.last_trade_price(scode)[0][0])*n*1.005 and not force_buy:
                self.log_lock.acquire()
                self.log.append(
                    "{}: no enough buying power for this portfolio to buy {} shares of {}".format(self.now(),n,scode)
                )
                self.log_lock.release()
                return 
            instrument = self.trader.instruments(scode)[0]
            order = self.trader.place_market_buy_order(instrument,n,time_in_force=time_in_force)
            if order is None:
                self.log_lock.acquire()
                self.log.append(
                    "{}: fail to place market buy order {} shares of {}".format(self.now(),n,scode)
                )
                self.log_lock.release()
                return
            self.queue_lock.acquire()
            self.queue.append([scode,order,0])
            self.queue_lock.release()
        t = Thread(target = market_buy_worker)
        self.threads.append(t)
        t.start()
        
    def market_sell(self,scode,n,time_in_force = 'gfd'):
        """
        sell stock with market order
        
        scode (str): symbol of stock
        n (int): shares to sell
        time_in_force (str): gfd ,gtc ,ioc ,fok or opg
        """
        def market_sell_worker():
            assert scode in self.portfolio_record.index
            self.portfolio_record_lock.acquire()
            if self.portfolio_record.loc[scode]["SHARES"] < n:
                self.portfolio_record_lock.release()
                self.log_lock.acquire()
                self.log.append(
                    "{}: no enough shares for this portfolio to sell {} shares of {}".format(self.now(),n,scode)
                )
                self.log_lock.release()
                return
            self.portfolio_record_lock.release()
            instrument = self.trader.instruments(scode)[0]
            order = self.trader.place_market_sell_order(instrument,n,time_in_force = time_in_force)
            if order.order is None:
                self.log_lock.acquire()
                self.log.append(
                    "{}: fail to place market sell order {} shares of {}".format(self.now(),n,scode)
                )
                self.log_lock.release()
                return
            self.queue_lock.acquire()
            self.queue.append([scode,order,0])
            self.queue_lock.release()
        t = Thread(target = market_sell_worker)
        self.threads.append(t)
        t.start()
    
    def stop_loss_buy(self,scode,stop_price,n,force_buy = False,time_in_force = 'gtc'):
        """
        buy stock with stop order
        
        scode (str): symbol of stock to buy
        stop_price (float): stop_price
        force_buy (bool): allow buying with buck up buying power
        time_in_force (str): gfd ,gtc ,ioc ,fok or opg
        """
        def stop_buy_worker():
            if self.bp < float(self.trader.last_trade_price(scode)[0][0])*n*1.005 and not force_buy:
                self.log_lock.acquire()
                self.log.append(
                    "{}: no enough buying power for this portfolio to buy {} shares of {}".format(self.now(),n,scode)
                )
                self.log_lock.release()
                return 
            instrument = self.trader.instruments(scode)[0]
            order = self.trader.place_stop_loss_buy_order(
                instrument,
                n,
                stop_price = stop_price,
                time_in_force = time_in_force
            )
            if order is None:
                self.log_lock.acquire()
                self.log.append(
                    "{}: fail to place stop loss buy order {} shares of {}".format(self.now(),n,scode)
                )
                self.log_lock.release()
                return
            self.queue_lock.acquire()
            self.queue.append([scode,order,0])
            self.queue_lock.release()
        t = Thread(target = stop_buy_worker)
        self.threads.append(t)
        t.start()
    def stop_loss_sell(self,scode,stop_price,n,time_in_force = 'gtc'):
        """
        sell stock with stop order
        
        scode (str): symbol of stock
        n (int): shares to sell
        time_in_force (str): gfd ,gtc ,ioc ,fok or opg
        """
        def stop_sell_worker():
            assert scode in self.portfolio_record.index
            self.portfolio_record_lock.acquire()
            if self.portfolio_record.loc[scode]["SHARES"] < n:
                self.portfolio_record_lock.release()
                self.log_lock.acquire()
                self.log.append(
                    "{}: no enough shares for this portfolio to sell {} shares of {}".format(self.now(),n,scode)
                )
                self.log_lock.release()
                return
            self.portfolio_record_lock.release()
            instrument = self.trader.instruments(scode)[0]
            order = self.trader.place_stop_loss_sell_order(
                instrument,
                n,
                stop_price = stop_price,
                time_in_force = time_in_force
            )
            if order.order is None:
                self.log_lock.acquire()
                self.log.append(
                    "{}: fail to place stop loss sell order {} shares of {}".format(self.now(),n,scode)
                )
                self.log_lock.release()
                return
            self.queue_lock.acquire()
            self.queue.append([scode,order,0])
            self.queue_lock.release()
        t = Thread(target = stop_sell_worker)
        self.threads.append(t)
        t.start()
        

    def limit_buy(self,scode,price,n,force_buy = False,time_in_force = 'gtc'):
        """
        buy stock with limit order
        
        scode (str): symbol of stock
        n (int): shares to sell
        time_in_force (str): gfd ,gtc ,ioc ,fok or opg
        """
        def limit_buy_worker():
            if self.bp < float(self.trader.last_trade_price(scode)[0][0])*n*1.005 and not force_buy:
                self.log_lock.acquire()
                self.log.append(
                    "{}: no enough buying power for this portfolio to buy {} shares of {}".format(self.now(),n,scode)
                )
                self.log_lock.release()
                return 
            instrument = self.trader.instruments(scode)[0]
            order = self.trader.place_limit_buy_order(
                instrument,
                n,
                price = price,
                time_in_force = time_in_force
            )
            if order is None:
                self.log_lock.acquire()
                self.log.append(
                    "{}: fail to place limit buy order {} shares of {}".format(self.now(),n,scode)
                )
                self.log_lock.release()
                return
            self.queue_lock.acquire()
            self.queue.append([scode,order,0])
            self.queue_lock.release()
        t = Thread(target = limit_buy_worker)
        self.threads.append(t)
        t.start()
        
    def limit_sell(self,scode,price,n,time_in_force = 'gtc'):
        """
        sell stock with limit order
        
        scode (str): symbol of stock
        n (int): shares to sell
        time_in_force (str): gfd ,gtc ,ioc ,fok or opg
        """
        def limit_sell_worker():
            assert scode in self.portfolio_record.index
            self.portfolio_record_lock.acquire()
            if self.portfolio_record.loc[scode]["SHARES"] < n:
                self.portfolio_record_lock.release()
                self.log_lock.acquire()
                self.log.append(
                    "{}: no enough shares for this portfolio to sell {} shares of {}".format(self.now(),n,scode)
                )
                self.log_lock.release()
                return
            self.portfolio_record_lock.release()
            instrument = self.trader.instruments(scode)[0]
            order = self.trader.place_limit_sell_order(
                instrument,
                n,
                price = price,
                time_in_force = time_in_force
            )
            if order.order is None:
                self.log_lock.acquire()
                self.log.append(
                    "{}: fail to place limit sell order {} shares of {}".format(self.now(),n,scode)
                )
                self.log_lock.release()
                return
            self.queue_lock.acquire()
            self.queue.append([scode,order,0])
            self.queue_lock.release()
        t = Thread(target = limit_sell_worker)
        self.threads.append(t)
        t.start()        
        
        
    def stop_limit_buy(self,scode,stop_price,n,force_buy = False,time_in_force = 'gtc'):
        """
        buy stock with stop limit order
        
        scode (str): symbol of stock to buy
        stop_price (float): stop_price
        force_buy (bool): allow buying with buck up buying power
        time_in_force (str): gfd ,gtc ,ioc ,fok or opg
        """
        def stop_limit_buy_worker():
            if self.bp < float(self.trader.last_trade_price(scode)[0][0])*n*1.005 and not force_buy:
                self.log_lock.acquire()
                self.log.append(
                    "{}: no enough buying power for this portfolio to buy {} shares of {}".format(self.now(),n,scode)
                )
                self.log_lock.release()
                return 
            instrument = self.trader.instruments(scode)[0]
            order = self.trader.place_stop_limit_buy_order(
                instrument,
                n,
                stop_price = stop_price,
                time_in_force = time_in_force
            )
            if order is None:
                self.log_lock.acquire()
                self.log.append(
                    "{}: fail to place stop loss buy order {} shares of {}".format(self.now(),n,scode)
                )
                self.log_lock.release()
                return
            self.queue_lock.acquire()
            self.queue.append([scode,order,0])
            self.queue_lock.release()
        t = Thread(target = stop_limit_buy_worker)
        self.threads.append(t)
        t.start()
    def stop_limit_sell(self,scode,stop_price,n,time_in_force = 'gtc'):
        """
        sell stock with stop limit order
        
        scode (str): symbol of stock
        n (int): shares to sell
        time_in_force (str): gfd ,gtc ,ioc ,fok or opg
        """
        def stop_limit_sell_worker():
            assert scode in self.portfolio_record.index
            self.portfolio_record_lock.acquire()
            if self.portfolio_record.loc[scode]["SHARES"] < n:
                self.portfolio_record_lock.release()
                self.log_lock.acquire()
                self.log.append(
                    "{}: no enough shares for this portfolio to sell {} shares of {}".format(self.now(),n,scode)
                )
                self.log_lock.release()
                return
            self.portfolio_record_lock.release()
            instrument = self.trader.instruments(scode)[0]
            order = self.trader.place_stop_limit_sell_order(
                instrument,
                n,
                stop_price = stop_price,
                time_in_force = time_in_force
            )
            if order.order is None:
                self.log_lock.acquire()
                self.log.append(
                    "{}: fail to place stop limit sell order {} shares of {}".format(self.now(),n,scode)
                )
                self.log_lock.release()
                return
            self.queue_lock.acquire()
            self.queue.append([scode,order,0])
            self.queue_lock.release()
        t = Thread(target = stop_limit_sell_worker)
        self.threads.append(t)
        t.start()
        
        

    def confirm_order(self,loop = False,gap_time = 5):
        """
        check whether submitted orders had been executed
        
        loop (bool): confirm once or keep on comfirming until signal received
        gap_time (float|int): pause between two confirms, in sec
        """
        gap_time = float(gap_time)
        if not self.confirm_signal:
            self.confirm_signal = True
        assert gap_time > 0
        def confirm_worker():
            if loop:
                self.log_lock.acquire()
                self.log.append("{}: confirm, start".format(self.get_time()))
                self.log_lock.release()
            while self.confirm_signal:
                sleep(gap_time)
                self.queue_lock.acquire()
                if not len(self.queue):
                    self.queue_lock.release()
                    continue
                scode,order,cc = self.queue.pop(0)
                self.queue_lock.release()
                d = order.check()
                if d['state'] in ['rejected','cancelled']:
                    self.log_lock.acquire()
                    self.log.append(
                        "{}: order ({},{} {} {} {}) {} for unknown reason".format(
                            self.get_time(),
                            scode,
                            d['quantity'],
                            d['trigger'],
                            d['type'],
                            d['side'],
                            d['state']
                        )
                    )
                    self.log_lock.release()
                    continue

                if not len(d['executions']):
                    self.queue_lock.acquire()
                    if cc >= self.cancel_count:
                        order.cancel()
                    else:
                        self.queue.append([scode,order,cc+1])
                    self.queue_lock.release()
                    continue

                ex_amount = float(d['executions'][0]['quantity'])
                ex_price = float(d['executions'][0]['price'])
                ex_side = d['side']

                if ex_side == 'sell':
                    ex_amount = - ex_amount

                self.trading_record_lock.acquire()
                self.trading_record.loc[d['executions'][0]['timestamp']] = [ex_side,scode,ex_price,abs(ex_amount),d['type']]
                self.trading_record_lock.release()

                self.portfolio_record_lock.acquire()
                if scode not in self.portfolio_record.index:
                    hold_amount = 0
                    avg_cost = 0
                else:
                    hold_amount = self.portfolio_record.loc[scode]['SHARES']
                    avg_cost = self.portfolio_record.loc[scode]['AVG_COST']

                total_cost = hold_amount*avg_cost + ex_price*ex_amount
                neo_shares = hold_amount + ex_amount
                self.bp = self.bp - total_cost
                if neo_shares == 0:
                    self.portfolio_record.drop(scode)
                    self.portfolio_record_lock.release()
                    continue

                neo_avg_cost = total_cost/neo_shares
                self.portfolio_record.loc[scode] = [neo_avg_cost,neo_shares]
                self.portfolio_record_lock.release()
                if not loop:
                    break
            if loop:
                self.log_lock.acquire()
                self.log.append("{}: confirm, end".format(self.get_time()))
        t = Thread(target = confirm_worker)
        t.start()
    def stop_confirm(self):
        self.confirm_signal = False
    def transfer_shares(self,oth = None,scode = None,amount = None,direction = 'to'):
        """
        transfer shares from one portfolio to another
        
        oth (Portfolio): another portfolio that shares the same trader as this one
        scode (str): symbol of stock to be transfered
        amount (int): shares of stock to be transfered
        direction (str): the direction of the transfer
        """
        assert oth.trader == self.trader
        assert scode is not None
        assert int(amount) > 0
        assert direction in ['from','to']
        amount = int(amount)
        if direction == 'from':
            oth.protfolio_record_lock.acquire()
            if (oth.protfolio_record.loc[scode]["SHARES"] < amount):
                oth.protfolio_record_lock.release()
                self.log_lock.acquire()
                self.log.append(
                    "{}: target portfolio doesnt have enough shares to transfer ({},{})".format(self.get_time(),scode,amount)
                )
                self.log_lock.release()
                return
            oth.protfolio_record.loc[scode]["SHARES"] -= amount
            transfer_price = oth.protfolio_record.loc[scode]["AVG_COST"]
            if oth.protfolio_record.loc[scode]["SHARES"] == 0:
                oth.protfolio_record.pop(scode)
            oth.protfolio_record_lock.release()
            
            self.portfolio_record_lock.acquire()
            if scode not in self.portfolio_record.index:
                original_avg_cost = 0
                original_hold = 0
            else:
                original_avg_cost = self.portfolio_record.loc[scode]["AVG_COST"]
                original_hold = self.portfolio_record.loc[scode]["SHARES"]
            neo_avg_cost = (original_avg_cost*original_hold + amount*transfer_price)/(original_hold+amount)
            self.portfolio_record.loc[scode] = [neo_avg_cost,original_hold+amount]
            self.portfolio_record_lock.release()
        if direction == 'to':
            other.transfer(self,scode,amount,direction = 'from')
            
    def transfer_buying_power(self,oth = None,amount = None,direction = 'to'):
        """
        transfer buying power from one portfolio to another
        oth (Portfolio): the other portfolio that shares the same trader as this one
        amount (float): the amount of money(in USD) to be transfered
        direction (str): the direction of this transfer
        """
        assert self.trader == oth.trader
        assert amount > 0
        assert direction in ['from','to']
        if direction == 'from':
            if oth.bp < amount:
                self.log_lock.acquire()
                self.log.append(
                    "{}: target portfolio doesnt have enough buying power to transfer ({})".format(self.get_time(),amount)
                )
                self.log_lock.release()
                return 
            oth.bp -= amount
            self.bp += amount
        if direction == 'to':
            other.transfer(self,amount,'from')
            
            
    def cancel_all_orders_in_queue(self):
        """
        cancel all orders in the queue that havent been executed yet
        """
        self.queue_lock.acquire()
        while len(self.queue):
            scode,order = self.queue.pop()
            order.cancel()
        self.queue_lock.release()
        
    def add_shares_from_pool(self,scode = None,n = None):
        """
        add share from account equity to portfolio
        currently, one should make sure the sum of # of shares in each portfolio is less than the 
        total holding shares manually when using it. A portfolio manager class will be created 
        to handle this later
        
        scode (str): symbol for the stock to be added
        n (int|float): amount to be added
        """
        owned = self.trader.securities_owned()['results']
        target_ins = self.trader.instruments(scode)[0]["url"]
        d = None
        for ins in owned:
            if ins['instrument'] == target_ins:
                d = ins
                break
        if d is None:
            self.log_lock.acquire()
            self.log.append(
                "{}: dont have {} in your pool".format(self.get_time(),scode)
            )
            self.log_lock.release()
            return
        owned_shares = float(d['quantity'])
        if n > owned_shares:
            self.log_lock.acquire()
            self.log.append(
                "{}: dont have enough shares of {} in you pool".format(self.get_time(),scode)
            )
            self.log_lock.release()
            return
        n_avg_cost = float(d['average_buy_price'])
        self.portfolio_record_lock.acquire()
        if scode not in self.portfolio_record:
            avg_cost = 0
            shares = 0
        else:
            avg_cost = self.portfolio_record.loc[scode]['AVG_COST']
            shares = self.portfolio_record.loc[scode]['SHARES']
        neo_shares = shares + n
        neo_cost = (avg_cost*shares + n_avg_cost*n)/neo_shares
        self.portfolio_record.loc[scode] = [neo_cost,neo_shares]
        self.portfolio_record_lock.release()
        
    def set_bp_HARD(self,bp):
        """
        force buying power to be bp, by default, the maximum buying power of a portforlio cannot exceed 70%
        of total buying power in case the buying order thread processed before a concurrent selling order.
        when a market buying order goes with a market selling order at the same time, set the force_buy parameter
        of market_buy if the buying power for buying order comes from the selling order's return.
        
        bp (float): buying power
        """
        total_bp = float(self.trader.get_account()['margin_balances']['unallocated_margin_cash'])
        self.bp = min(bp,total_bp)
        
    def is_market_open(self):
        """
        check if a market is open,
        
        have no clue what timezone did robinhood use for market hour,
        official api says it is US/Eastern, but the market hour returned from api is obviously not US/Eastern
        
        so to check the current time with the timezone robinhood used,
        a buy order will be placed to see the created time returned by api
        
        the placed order will normally be rejected or this method will cancel it immediately.
        
        shoud not call this method too frequently
        """
        u = self.trader.place_stop_limit_buy_order(
            instrument = self.trader.instruments('BAC')[0],
            quantity=1,
            stop_price=0.01
        )
        now = datetime.datetime.strptime(u.check()['created_at'],"%Y-%m-%dT%H:%M:%S.%fZ")
        u.cancel()
        y = now.year
        m = now.month
        d = now.day
        info = self.trader.session.get('https://api.robinhood.com/markets/XNAS/hours/{}-{}-{}/'.format(y,m,d)).json()
        if not info['is_open']:
            return False
        opens_at = datetime.datetime.strptime(info['opens_at'],"%Y-%m-%dT%H:%M:%SZ")
        closes_at = datetime.datetime.strptime(info['closes_at'],"%Y-%m-%dT%H:%M:%SZ")
        return now<=closes_at and now >=opens_at
        
        
    def shares_owned(self,scode):
        """
        get number of shares of a stock in this portfolio
        
        scode (str): symbol of stock
        """
        self.portfolio_record_lock.acquire()
        if scode not in self.portfolio_record.index:
            self.portfolio_record_lock.release()
            return 0
        res = self.portfolio_record.loc[scode]["SHARES"]
        self.portfolio_record_lock.release()
        return res
        
    def save(self,savdir = None,root_name = ''):
        """
        save portfolio to files
        """
        for t in self.threads:
            t.join()
        if savdir is None:
            savdir = self.name
        
        fdir = root_name+savdir+'/'
        if not os.path.exists(fdir):
            os.mkdir(fdir)
        self.trading_record.to_csv(fdir+"trading.csv")
        self.portfolio_record.to_csv(fdir+"portfolio.csv")
        pd.DataFrame([[self.bp]]).to_csv(fdir+"bp")
        with open(fdir+"log{}.log".format(self.get_time()).replace(' ','').replace(':','.'),'w') as f:
            for log in self.log:
                f.write(log+"\n")
                
    def load(self,savdir = None,root_name = ''):
        """
        load portfolio info from files
        """
        if savdir is None:
            savdir = self.name
        fdir = root_name + savdir + '/'
        assert os.path.exists(fdir)
        self.trading_record = pd.DataFrame.from_csv(fdir+"trading.csv")
        self.portfolio_record = pd.DataFrame.from_csv(fdir+"portfolio.csv")
        self.bp = pd.DataFrame.from_csv(fdir+"bp").values[0][0]
        
        
                
        

In [146]:
class PF:
    def __init__(
        self,
        scodes,
        Rf=0.01/365,
        period = 100,
        apikey = "rixaXs71r2KgkmPHW9jZ",
        update_now = False
    ):
        self.scodes = scodes
        self.Rf = Rf
        self.period = period
        self.apikey = apikey
        self.data = pd.DataFrame()
        if update_now:
            self.update()
        self.gmv = None
        self.mve = None
    def update(self,get_data = True):
        if get_data:
            quandl.ApiConfig.api_key = self.apikey
            self.data = pd.DataFrame()
            for scode in self.scodes:
                cl = quandl.get("EOD/"+scode.replace(".","_"))["Close"]
                self.data[scode] = ((cl-cl.shift(1))/cl.shift(1)).iloc[-self.period:]
                print(scode,end = ' ')
            quandl.ApiConfig.api_key = None
        self.cov = self.data.cov()
        self.std = self.data.std(ddof=0)
        self.mean = self.data.mean()
    def global_variance(self,w):
        assert len(w) == len(self.scodes)
        res = np.dot(np.matmul(w,self.cov.values),w)
        return np.sqrt(res)
    def sharpe_ratio(self,w):
        assert len(w) == len(self.scodes)
        return (np.dot(w,self.mean.values)-self.Rf)/self.global_variance(w)
    def GMV(self,no_short = True,tol = 1e-20,maxiter = 600):
        w0 = np.array([1.0/len(self.scodes)]*len(self.scodes))
        paras = {
            'fun':lambda x:self.global_variance(x),
            'x0':w0,
            'method':'SLSQP',
            'constraints':{
                'type':'eq',
                'fun':(lambda x:sum(x)-1)
            },
            'tol':tol,
            'options':{
                "maxiter" : maxiter
            }
        }
        if no_short:
            paras['bounds'] = [(0,1) for i in range(len(self.scodes))]
        gmv = minimize(**paras)
        if gmv.message != 'Optimization terminated successfully.':
            print("Error during GMV minimize:")
            print(gmv.message)
        self.Wgmv = gmv.x
        self.Rgmv = np.dot(gmv.x,self.mean)
        self.Vgmv = gmv.fun
        self.gmv = gmv
    def MVE(self,no_short = True,tol = 1e-20,maxiter = 600):
        w0 = np.array([1.0/len(self.scodes)]*len(self.scodes))
        paras = {
            'fun':lambda x:-self.sharpe_ratio(x),
            'x0':w0,
            'method':'SLSQP',
            'constraints':{
                'type':'eq',
                'fun':(lambda x:sum(x)-1)
            },
            'tol':tol,
            'options':{
                'maxiter' : maxiter
            }
        }
        if no_short:
            paras['bounds'] = [(0,1) for i in range(len(self.scodes))]
        mve = minimize(**paras)
        if mve.message != 'Optimization terminated successfully.':
            print("Error during MVE minimize:")
            print(mve.message)
        self.Wmve = mve.x
        self.Rmve = np.dot(mve.x,self.mean)
        self.Vmve = self.global_variance(mve.x)
        self.mve = mve
    def GMV_MVE_cov(self):
        if self.gmv == None:
            self.GMV()
        if self.mve == None:
            self.MVE()
        wmve = self.Wmve.reshape(-1,1)
        wgmv = self.Wgmv.reshape(-1,1)
        U = np.matmul(wgmv,wmve.T)
        return np.dot(U.flatten(),self.cov.values.flatten())
    def get_pf(self,gmvw = None,mvew = None):
        if gmvw == None:
            gmvw = 1-mvew
        else:
            mvew = 1-gmvw
        cov = self.GMV_MVE_cov()
        r = gmvw*self.Rgmv + mvew*self.Rmve
        d = np.sqrt(gmvw**2*self.Vgmv**2 + mvew**2*self.Vmve**2 + 2*mvew*gmvw*cov)
        return d,r
        
        
    def analog(self,gmvw = None,mvew = None,no_short = False,tol = 1e-20,maxiter = 600):
        if gmvw == None:
            gmvw = 1-mvew
        else:
            mvew = 1-gmvw
            
        DT = pd.DataFrame()
        quandl.ApiConfig.api_key = self.apikey
        for scode in self.scodes:
            cl = quandl.get("EOD/"+scode.replace(".","_"))['Close']
            print(len(cl))
            DT[scode] = (cl-cl.shift(1))/cl.shift(1)
        DT = DT.dropna()
        quandl.ApiConfig.api_key = None
        ws = [np.zeros(len(self.scodes))]
        rs = []
        for i in range(len(DT)-self.period):
            self.data = DT.iloc[i:i+self.period]
            self.update(get_data = False)
            self.GMV(no_short = no_short,maxiter = maxiter,tol=tol)
            self.MVE(no_short = no_short,maxiter = maxiter,tol=tol)
        
            rs.append(np.dot(ws[-1],self.data.iloc[-1].values))
            ws.append(gmvw*self.Wgmv + mvew*self.Wmve)
        return rs,ws

In [12]:
trader = Robinhood()

In [13]:
trader.login('hang2','zhhISbest2')

True

In [139]:
class AT:
    def __init__(
        self,
        robin_un,
        robin_pd,
        scodes,
        no_short = True,
        minimize_tol = 1e-12,
        minimize_maxiter = 600,
        daily_Rf = 0.01/365,
        obv_period = 100,
        api_key = "rixaXs71r2KgkmPHW9jZ",
        iniFund = None,
        iniPos = None
    ):
        self.trader = Robinhood()
        self.trader.login(username = robin_un,password = robin_pd)
        print("initializing...")
        self.pf = PF(
            scodes = scodes,
            Rf = daily_Rf,
            period=obv_period,
            apikey = api_key,
            update_now = True
        )
        self.tol = minimize_tol
        self.maxiter = minimize_maxiter
        self.no_short = no_short
    def get_position(self):
        d = self.trader.securities_owned()
        return {json.loads(urlopen(x['instrument']).read().decode("UTF-8"))['symbol']:{
            'average_buy_price':float(x['average_buy_price']),
            'quantity':float(x['quantity'])
        } for x in d['results']}
    def realloc(self,wgmv):
        self.pf.GMV(no_short = self.no_short,tol = self.tol,maxiter = self.maxiter)
        self.pf.MVE(no_short = self.no_short,tol = self.tol,maxiter = self.maxiter)
        last_price = np.array([d['last_trade_price'] for d in self.trader.quotes_data(self.pf.scodes)],dtype=np.float32)
        position = self.get_position()
        total = trader.equity()
        current_w = np.array(
            [0 if scode not in position else position[scode]['average_buy_price']*position[scode]['quantity']/total for scode in self.pf.scodes]
        )
        target_w = wgmv*self.pf.Wgmv + (1-wgmv)*self.pf.Wmve
        
        w_diff = target_w - current_w
        return (w_diff*total/last_price).astype(int)
    

In [147]:
at = AT(
    robin_un = "hang2",
    robin_pd = "zhhISbest2",
    scodes = ["RH","BABA","PCRX","ADBE","IBKR","F","T","CMCSA","NVDA","CNX","NTES","QCRH","FFBC","YRD","BRK.B"],
    obv_period = 300
)

initializing...
RH BABA PCRX ADBE IBKR F T CMCSA NVDA CNX NTES QCRH FFBC YRD BRK.B 

In [148]:
ww = at.realloc(0.4)

In [143]:
at.get_position()

{'BRK.B': {'average_buy_price': 193.4132, 'quantity': 22.0},
 'FFBC': {'average_buy_price': 26.299, 'quantity': 20.0},
 'RH': {'average_buy_price': 95.3372, 'quantity': 23.0},
 'YRD': {'average_buy_price': 40.3688, 'quantity': 24.0}}

In [10]:
trader = Robinhood()

NameError: name 'Robinhood' is not defined

In [38]:
trader.login(username='hang2',password="zhhISbest2")

True

In [40]:
yrd = trader.instruments("YRD")[0]

In [20]:
trader.quote_data("YRD")

{'adjusted_previous_close': '45.4400',
 'ask_price': '44.8800',
 'ask_size': 700,
 'bid_price': '43.9800',
 'bid_size': 100,
 'has_traded': True,
 'instrument': 'https://api.robinhood.com/instruments/78bde958-d99c-40a8-90c5-904fd466977a/',
 'last_extended_hours_trade_price': '44.6600',
 'last_trade_price': '44.6600',
 'last_trade_price_source': 'consolidated',
 'previous_close': '45.4400',
 'previous_close_date': '2018-01-04',
 'symbol': 'YRD',
 'trading_halted': False,
 'updated_at': '2018-01-06T01:00:00Z'}

In [41]:
r = trader.place_buy_order(yrd,1)

In [1]:
import numpy as np

In [7]:
f = np.array([False])

In [8]:
f.__getattribute__("all")()

False

In [2]:
import quandl

In [3]:
quandl.ApiConfig.api_key = "rixaXs71r2KgkmPHW9jZ"

In [7]:

a = quandl.get("EOD/MMM",rows = 10)


b = quandl.get("EOD/MMM")


In [8]:
a

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,,,,,,,,
2017-12-29,236.48,237.3100,235.370,235.37,1343878.0,0.0,1.0,236.48,237.3100,235.370,235.37,1343878.0
2018-01-02,235.78,237.0700,232.805,235.64,2930951.0,0.0,1.0,235.78,237.0700,232.805,235.64,2930951.0
2018-01-03,235.07,235.7300,233.290,235.63,2193713.0,0.0,1.0,235.07,235.7300,233.290,235.63,2193713.0
2018-01-04,237.00,239.4400,236.470,238.71,2243101.0,0.0,1.0,237.00,239.4400,236.470,238.71,2243101.0
2018-01-05,238.65,240.9000,237.740,240.57,1835609.0,0.0,1.0,238.65,240.9000,237.740,240.57,1835609.0
2018-01-08,239.38,240.9400,239.180,239.79,1869025.0,0.0,1.0,239.38,240.9400,239.180,239.79,1869025.0
2018-01-09,239.60,241.7845,239.340,241.28,1695114.0,0.0,1.0,239.60,241.7845,239.340,241.28,1695114.0
2018-01-10,241.00,242.5700,240.030,241.14,1640853.0,0.0,1.0,241.00,242.5700,240.030,241.14,1640853.0
2018-01-11,240.74,242.3400,239.750,242.31,1487650.0,0.0,1.0,240.74,242.3400,239.750,242.31,1487650.0


In [9]:
b.tail()

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,,,,,,,,
2018-01-08,239.38,240.9400,239.18,239.79,1869025.0,0.0,1.0,239.38,240.9400,239.18,239.79,1869025.0
2018-01-09,239.60,241.7845,239.34,241.28,1695114.0,0.0,1.0,239.60,241.7845,239.34,241.28,1695114.0
2018-01-10,241.00,242.5700,240.03,241.14,1640853.0,0.0,1.0,241.00,242.5700,240.03,241.14,1640853.0
2018-01-11,240.74,242.3400,239.75,242.31,1487650.0,0.0,1.0,240.74,242.3400,239.75,242.31,1487650.0
2018-01-12,243.07,246.0000,242.50,244.47,1955097.0,0.0,1.0,243.07,246.0000,242.50,244.47,1955097.0


In [53]:
import json
from urllib.request import urlopen

'{"min_tick_size":null,"type":"stock","splits":"https:\\/\\/api.robinhood.com\\/instruments\\/a4f0cca4-79dc-4297-9c02-5bce1909cd4b\\/splits\\/","margin_initial_ratio":"0.5000","url":"https:\\/\\/api.robinhood.com\\/instruments\\/a4f0cca4-79dc-4297-9c02-5bce1909cd4b\\/","quote":"https:\\/\\/api.robinhood.com\\/quotes\\/BRK.B\\/","tradability":"tradable","bloomberg_unique":"EQ0010019300003000","list_date":"1998-06-29","name":"Berkshire Hathaway Class B","symbol":"BRK.B","fundamentals":"https:\\/\\/api.robinhood.com\\/fundamentals\\/BRK.B\\/","state":"active","country":"US","day_trade_ratio":"0.2500","tradeable":true,"maintenance_ratio":"0.2500","id":"a4f0cca4-79dc-4297-9c02-5bce1909cd4b","market":"https:\\/\\/api.robinhood.com\\/markets\\/XNYS\\/","simple_name":"Berkshire Hathaway"}'

In [63]:
{
    json.loads(urlopen(x['instrument'])
               .read()
               .decode("UTF-8"))['symbol']:{
        'average_buy_price':x['average_buy_price'],
        'quantity':x['quantity']
    } for x in d['results']
}

{'BRK.B': {'average_buy_price': '193.4132', 'quantity': '22.0000'},
 'FFBC': {'average_buy_price': '26.2990', 'quantity': '20.0000'},
 'RH': {'average_buy_price': '95.3372', 'quantity': '23.0000'},
 'YRD': {'average_buy_price': '40.3688', 'quantity': '24.0000'}}

In [50]:
import Robinhood as rh

In [ ]:
rh.exceptions.